In [27]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchinfo import summary
from torchvision import datasets, transforms

In [28]:
# 1. Configuration and Data Loading
BATCH_SIZE = 64
transform = transforms.Compose([
    transforms.ToTensor(),
    # Normalize with MNIST mean (0.1307) and std (0.3081)
    transforms.Normalize((0.1307,), (0.3081,))
])

In [29]:
# Download and load training data
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)

# Download and load test data
test_dataset = datasets.MNIST('./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=1000, shuffle=False)

In [30]:
# Use GPU if available, otherwise CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cpu')

In [ ]:
# 2. Define the Neural Network Model
class SimpleNN(nn.Module):
    def __init__(self):
        super(SimpleNN, self).__init__()

        self.conv1 = nn.Conv2d(in_channels=1, out_channels=8, kernel_size=3)  #26 26
        self.pool1 = nn.MaxPool2d(kernel_size=2)  # 13 13

        self.conv2 = nn.Conv2d(8, 16, kernel_size=3)  #11 11
        # self.pool2 = nn.AdaptiveAvgPool2d(output_size=(3, 3))
        self.pool2 = nn.MaxPool2d(kernel_size=3)

        self.fc3 = nn.Linear(16 * 3 * 3, 10)

    def forward(self, x):
        # Flatten the input image from 28x28 to a 784-element vector
        x = F.relu(self.conv1(x))
        x = self.pool1(x)

        x = F.relu(self.conv2(x))
        x = self.pool2(x)

        x = x.view(-1, 16 * 3 * 3)
        x = self.fc3(x)
        # Apply log softmax for the final output (often used with NLLLoss, which is part of CrossEntropyLoss)
        return F.log_softmax(x, dim=1)


model = SimpleNN().to(device)

In [ ]:
# 3. Define Loss and Optimizer
criterion = nn.NLLLoss()  # Negative Log-Likelihood Loss (suitable for log_softmax output)
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [43]:
summary(model)

Layer (type:depth-idx)                   Param #
SimpleNN                                 --
├─Conv2d: 1-1                            80
├─MaxPool2d: 1-2                         --
├─Conv2d: 1-3                            1,168
├─AdaptiveAvgPool2d: 1-4                 --
├─Linear: 1-5                            1,450
Total params: 2,698
Trainable params: 2,698
Non-trainable params: 0

In [ ]:
from datetime import datetime

writer = SummaryWriter(f'runs/init_{datetime.now().strftime("%Y_%m_%d-%H_%M_%S")}')

In [ ]:
# 4. Training Loop
NUM_EPOCHS = 5

for epoch in range(NUM_EPOCHS):
    model.train()
    running_loss = 0
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)

        optimizer.zero_grad()  # Clear gradients
        output = model(data)  # Forward pass
        loss = criterion(output, target)  # Calculate loss
        loss.backward()  # Backward pass (calculate gradients)
        optimizer.step()  # Update weights

        running_loss += loss.item()

        if batch_idx % 100 == 0:
            print(f'Train Epoch: {epoch + 1}/{NUM_EPOCHS} [{batch_idx * len(data)}/{len(train_loader.dataset)} '
                  f'({100. * batch_idx / len(train_loader):.0f}%)]\tLoss: {loss.item():.6f}')
    avg_train_loss = running_loss / len(train_loader)
    writer.add_scalar('Loss/Train', avg_train_loss, epoch)

writer.flush()
writer.close()

Train Epoch: 2/5 [25600/60000 (43%)]	Loss: 0.152865
Train Epoch: 2/5 [32000/60000 (53%)]	Loss: 0.079841
Train Epoch: 2/5 [38400/60000 (64%)]	Loss: 0.162038
Train Epoch: 2/5 [44800/60000 (75%)]	Loss: 0.097246
Train Epoch: 2/5 [51200/60000 (85%)]	Loss: 0.119199
Train Epoch: 2/5 [57600/60000 (96%)]	Loss: 0.052596
Train Epoch: 3/5 [0/60000 (0%)]	Loss: 0.150450
Train Epoch: 3/5 [6400/60000 (11%)]	Loss: 0.050109
Train Epoch: 3/5 [12800/60000 (21%)]	Loss: 0.048268
Train Epoch: 3/5 [19200/60000 (32%)]	Loss: 0.121533
Train Epoch: 3/5 [25600/60000 (43%)]	Loss: 0.135786
Train Epoch: 3/5 [32000/60000 (53%)]	Loss: 0.086203
Train Epoch: 3/5 [38400/60000 (64%)]	Loss: 0.131176
Train Epoch: 3/5 [44800/60000 (75%)]	Loss: 0.148006
Train Epoch: 3/5 [51200/60000 (85%)]	Loss: 0.088567
Train Epoch: 3/5 [57600/60000 (96%)]	Loss: 0.043053
Train Epoch: 4/5 [0/60000 (0%)]	Loss: 0.052097
Train Epoch: 4/5 [6400/60000 (11%)]	Loss: 0.106025
Train Epoch: 4/5 [12800/60000 (21%)]	Loss: 0.051528
Train Epoch: 4/5 [19200/

In [ ]:
model.eval()

SimpleNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): AdaptiveAvgPool2d(output_size=(3, 3))
  (fc3): Linear(in_features=144, out_features=10, bias=True)
)

In [ ]:
model.cpu()

SimpleNN(
  (conv1): Conv2d(1, 8, kernel_size=(3, 3), stride=(1, 1))
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(8, 16, kernel_size=(3, 3), stride=(1, 1))
  (pool2): AdaptiveAvgPool2d(output_size=(3, 3))
  (fc3): Linear(in_features=144, out_features=10, bias=True)
)

In [ ]:
# tensorboard --logdir=runs

In [ ]:
# 5. Testing the Model
test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.argmax(dim=1, keepdim=True)
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(test_loader.dataset)
accuracy = 100. * correct / len(test_loader.dataset)

print(f'\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(test_loader.dataset)} '
      f'({accuracy:.2f}%)\n')


Test set: Average loss: 0.0872, Accuracy: 9744/10000 (97.44%)



In [40]:
# Save the model (optional)
torch.save(model.state_dict(), "mnist_simple_nn.pth")

In [41]:
def export_onnx(model, onnx_path="dynamic_cnn.onnx"):
    dummy = torch.randn(1, 1, 28, 28)

    with torch.inference_mode():
        torch.onnx.export(
            model,
            dummy,
            onnx_path,
            opset_version=18,
            dynamo=False,
            export_params=True,
            do_constant_folding=True,
            input_names=["input"],
            output_names=["output"],
            dynamic_axes={
                "input": {0: "batch", 2: "height", 3: "width"},
                "output": {0: "batch"},
            },
        )

In [42]:
onnx_path = "model.onnx"
export_onnx(model, onnx_path)

C:\Users\nick\AppData\Local\Temp\ipykernel_20084\3476701642.py:5: DeprecationWarning: You are using the legacy TorchScript-based ONNX export. Starting in PyTorch 2.9, the new torch.export-based ONNX exporter will be the default. To switch now, set dynamo=True in torch.onnx.export. This new exporter supports features like exporting LLMs with DynamicCache. We encourage you to try it and share feedback to help improve the experience. Learn more about the new export logic: https://pytorch.org/docs/stable/onnx_dynamo.html. For exporting control flow: https://pytorch.org/tutorials/beginner/onnx/export_control_flow_model_to_onnx_tutorial.html.
  torch.onnx.export(


SymbolicValueError: Unsupported: ONNX export of operator adaptive_avg_pool2d, input size not accessible. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues  [Caused by the value 'input.4 defined in (%input.4 : Float(*, 16, *, *, strides=[250000, 15625, 125, 1], requires_grad=0, device=cpu) = onnx::Relu(%19), scope: __main__.SimpleNN:: # D:\programming\python\2025-NN-AI\.venv\lib\site-packages\torch\nn\functional.py:1697:0
)' (type 'Tensor') in the TorchScript graph. The containing node has kind 'onnx::Relu'.] 
    (node defined in D:\programming\python\2025-NN-AI\.venv\lib\site-packages\torch\nn\functional.py(1697): relu
C:\Users\nick\AppData\Local\Temp\ipykernel_20084\666256591.py(20): forward
D:\programming\python\2025-NN-AI\.venv\lib\site-packages\torch\nn\modules\module.py(1765): _slow_forward
D:\programming\python\2025-NN-AI\.venv\lib\site-packages\torch\nn\modules\module.py(1786): _call_impl
D:\programming\pyt

In [ ]:
import onnx

onnx_model = onnx.load(onnx_path)
onnx.checker.check_model(onnx_model)  # Check for model correctness

print("Model inputs:")
for input in onnx_model.graph.input:
    print(f"Input name: {input.name}")
    print(
        f"Input shape: {[(d.dim_value if d.HasField('dim_value') else d.dim_param) for d in input.type.tensor_type.shape.dim]}")

In [ ]:
# Take a look at model graph
# https://netron.app/